In [ ]:
# -*- coding:utf-8 -*-

import cv2
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff


def FouriesTransformAndFilterBackToImg(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    #magnitude_spectrum = 20 * np.log(np.abs(fshift))

    rows, cols = img.shape
    crow,ccol = rows/2 , cols/2
    fshift[crow-800:crow+800, ccol-3000:ccol+3000] = 0
    f_ishift = np.fft.ifftshift(fshift)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)

    return img_back

file_2015 = './Match/20170905_preliminary/preliminary/quickbird2015.tif'
file_2017 = './Match/20170905_preliminary/preliminary/quickbird2017.tif'

sat_2015 = tiff.imread(file_2015).transpose([1, 2, 0])
img_outred_2015 = sat_2015[:,:,3]
img_outred_2015 = (img_outred_2015 - np.mean(img_outred_2015)) / np.std(img_outred_2015)
img_outred_2015 = img_outred_2015 * 255
sat_2015 = None

img_2015_lowpass = FouriesTransformAndFilterBackToImg(img_outred_2015)

tiff.imsave('./Match/Out/img_2015_lowpass.tiff', img_2015_lowpass)

img_outred_2015 = None


sat_2017 = tiff.imread(file_2017).transpose([1, 2, 0])
img_outred_2017 = sat_2017[:,:,3]
img_outred_2017 = (img_outred_2017 - np.mean(img_outred_2017)) / np.std(img_outred_2017)
img_outred_2017 = img_outred_2017 * 255
sat_2017 = None

img_2017_lowpass = FouriesTransformAndFilterBackToImg(img_outred_2017)

tiff.imsave('./Match/Out/img_2017_lowpass.tiff', img_2017_lowpass)
img_outred_2015 = None

out_a = img_2017_lowpass - img_2015_lowpass

img_2015_lowpass = None
img_2017_lowpass = None

# map to min max 255
min_out_a = np.min(out_a)
max_out_a = np.max(out_a)
out_a = ( out_a - min_out_a ) * 255.0 / max_out_a

"""
out_a_2 = np.square(out_a)
out_a_2 = np.sqrt(out_a_2)
thread_mean = np.mean(out_a_2)  # 均值，改为中位数，试试
thread_sigma = np.std(out_a_2)
thread_median = np.median(out_a_2)

print('mean is ' + str(thread_mean))
print('mean is ' + str(thread_median))

thread = thread_mean + 3 * thread_sigma
tf = out_a_2 >= thread
out_b = tf.astype(np.uint8)
tiff.imsave('./Match/Out/sub920_mean_3sigma.tiff', out_b)
"""

## 高斯模糊啊
blur = cv2.GaussianBlur(out_a,(11,11),1)

thread_mean = np.mean(blur)  # 均值，改为中位数，试试
thread_sigma = np.std(blur)
thread_median = np.median(blur)

print('mean is ' + str(thread_mean))
print('median is ' + str(thread_median))

thread = thread_mean + thread_sigma
tf = blur >= thread
out_b = tf.astype(np.uint8)
tiff.imsave('./Match/Out/sub920_gaussin_mean_1sigma.tiff', out_b)
